# Constrained Food Recommender

In this assignment, you will implement both Content Based and Collaborative Filtering Recommenders and backtracking search (or local search) on your own

100% finished homework should contain EDA, Item and User profiles generation, Content-Based Recommender, Collaborative Filtering Recommender, and soluton to CSP problem of assigning recommendations to brekfast, lunch and dinner.

## Data loading

You will work with subset of [Academic Yelp Dataset](https://www.kaggle.com/yelp-dataset/yelp-dataset) containing list of restaurants in **yelp_business.csv** and reviews of the users in **yelp_reviews.parquet**

In [1]:
import pandas as pd

df_yelp_business = pd.read_csv("yelp_business.csv").drop(columns=["Unnamed: 0"])
df_yelp_reviews = pd.read_parquet("yelp_reviews.parquet")

# Leave only users with at least 3 reviews
users_count = df_yelp_reviews.groupby("user_id").count()[["business_id"]] 
users_to_use = users_count[users_count["business_id"] > 2]
df_yelp_reviews = df_yelp_reviews[df_yelp_reviews["user_id"].isin(users_to_use.index)]

## Exploratory data analysis

Here you will explore the data to find out what is the distribution of business categories, hours, places, user reviews, etc.

This step is needed to proceed later with item and user profiling and to clean your data if there are duplicates (e.g. duplicated reviews, the same businesses under different ids, categories tags which are highly correlated) or some artifacts not related to the main task.

(5 points)

In [68]:
df_yelp_business.categories.head(100)

0     Ethnic Food, Food Trucks, Specialty Food, Impo...
1            Food, Restaurants, Grocery, Middle Eastern
2          Japanese, Fast Food, Food Court, Restaurants
3      Food, Pretzels, Bakeries, Fast Food, Restaurants
4                       Mexican, Restaurants, Fast Food
                            ...                        
95                         Restaurants, Spanish, French
96    Fast Food, Chicken Shop, Restaurants, Chicken ...
97                    Restaurants, Steakhouses, Seafood
98    Coffee & Tea, Food, Juice Bars & Smoothies, De...
99                                   Delis, Restaurants
Name: categories, Length: 100, dtype: object

In [3]:
df_yelp_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
4,IS4cv902ykd8wj1TR0N3-A,0,2017-01-14 21:56:57,0,6TdNDKywdbjoTkizeMce8A,4,happy day finally canes near casa yes others g...,0,UgMW8bLE0QMJDCkQ1Ax5Mg
6,Pthe4qk5xh4n-ef-9bvMSg,0,2015-11-05 23:11:05,0,ZayJ1zWyWgY9S_TRLT_y9Q,5,really good place simple decor amazing food gr...,1,aq_ZxGHiri48TUXJlpRkCQ
9,Ws8V970-mQt2X9CwCuT5zw,1,2009-10-13 04:16:41,0,z4BCgTkfNtCu4XY5Lp97ww,4,twice nice laid back tried weekend southern me...,3,jOERvhmK6_lo_XGUBPws_w
16,d4qwVw4PcN-_2mK2o1Ro1g,0,2015-02-02 06:28:00,0,bVTjZgRNq8ToxzvtiVrqMA,1,10pm super bowl sunday already closed weak won...,0,2hRe26HSCAWbFRn5WChK-Q
22,9Jo1pu0y2zU6ktiwQm6gNA,20,2016-12-04 03:15:21,19,sgTnHfeaEvyOoWX4TCgkuQ,4,coconut fish cafe fantastic five stars fish ca...,24,A0j21z2Q1HGic7jW6e9h7A


In [4]:
df_yelp_reviews = df_yelp_reviews.drop_duplicates()
df_yelp_business = df_yelp_business.drop_duplicates()

In [5]:
longitude = df_yelp_business.longitude.copy()
longitude = longitude.astype(str)
latitude = df_yelp_business.latitude.copy()
latitude = latitude.astype(str)

In [6]:
latitude

0        40.11044570000001
1                35.194894
2               43.8204923
3       33.602821999999996
4       36.099737899999994
               ...        
5743         51.0730357311
5744            43.7310425
5745            43.7053513
5746     41.48468629999999
5747    41.396168599999996
Name: latitude, Length: 5748, dtype: object

In [7]:
df_yelp_business['full_name'] = df_yelp_business['name'] + ":" + longitude + ":" + latitude

In [8]:
df_yelp_business.full_name.value_counts().max() # So there are no exactly same restaurants

1

In [9]:
df_yelp_business = df_yelp_business.drop(columns=['full_name'])

## Building recommender

First of all you should process user reviews to get the utility matrix containing ratings for users and businesses. There will be a lot of 0 in this matrix and it is better to store such matrices in the specialized data structure for sparce matrices. However, your working dataset is relatively small and we can use simple **pd.DataFrame** to proceed

In [10]:
def create_utility_matrix(reviews: pd.DataFrame, business: pd.DataFrame) -> pd.DataFrame:
    business_ids = business["business_id"].unique()
    users = reviews["user_id"].unique()

    ut_matrix = pd.DataFrame(0, columns=business_ids, index=users)
    for _, review in reviews.iterrows():
        ut_matrix.loc[review["user_id"], review["business_id"]] = review["stars"]
    
    min_star = 1
    max_star = 5
    ut_matrix[ut_matrix == 3] = 3.000001 # It's needed for at_least_one_visited_place_constraint, and as
    # there are not many such values, it will not affect main code
    ut_matrix[ut_matrix == 0] = 3 # It's average review, and when we normalize it transforms to 0
    ut_matrix = 2 * (ut_matrix - min_star) / (max_star - min_star) - 1
    
    return ut_matrix

df_utility_matrix = create_utility_matrix(df_yelp_reviews, df_yelp_business)

In [11]:
df_utility_matrix.min(),df_utility_matrix.max()

(pQeaRpvuhoEqudo3uymHIQ    0.0
 CsLQLiRoafpJPJSkNX2h5Q    0.0
 lu7vtrp_bE9PnxWfA8g4Pg    0.0
 vjTVxnsQEZ34XjYNS-XUpA    0.0
 fnZrZlqW1Z8iWgTVDfv_MA   -1.0
                          ... 
 gp_bu7Ah81qaBY3M0Leffw    0.0
 PUKOr5bEI87TVHjwijT1xw    0.0
 zV38gkkEeJ4cVRlSWWQTfQ   -0.5
 H1j34TgbrVZkxeww9xlJTw   -0.5
 F8M0IukXQqR50IRyocRQbg   -1.0
 Length: 5748, dtype: float64,
 pQeaRpvuhoEqudo3uymHIQ    0.000000e+00
 CsLQLiRoafpJPJSkNX2h5Q    5.000000e-07
 lu7vtrp_bE9PnxWfA8g4Pg    5.000000e-01
 vjTVxnsQEZ34XjYNS-XUpA    1.000000e+00
 fnZrZlqW1Z8iWgTVDfv_MA    1.000000e+00
                               ...     
 gp_bu7Ah81qaBY3M0Leffw    5.000000e-01
 PUKOr5bEI87TVHjwijT1xw    0.000000e+00
 zV38gkkEeJ4cVRlSWWQTfQ    1.000000e+00
 H1j34TgbrVZkxeww9xlJTw    1.000000e+00
 F8M0IukXQqR50IRyocRQbg    5.000000e-07
 Length: 5748, dtype: float64)

## Content-Based Recommender

In [12]:
import ast
def build_business_profiles(business: pd.DataFrame) -> pd.DataFrame:
    # TODO: Feature engineering (10 points)
    business = business.copy()
    columns_to_save = ['is_open','review_count','stars']
    all_categories = set()
    for categories in business.categories:
        categories = [x.lower().strip() for x in categories.split(',')]
        all_categories = all_categories | set(categories)
    for category in all_categories:
        business.insert(1,category,0,True)
    i = 0
    for categories in business.categories:
        categories = [x.lower().strip() for x in categories.split(',')]
        for category in set(categories):
            business.at[i,category] = 1
        i+=1
    # I will not work with attributes, as they contain to many information, which could lead to overfit,
    # but I decide to work with all categories, as it's hard for our model to work with text, I transform
    # all data using one-hot encoding, 
    # also I decide to remove all data contain coordinates(address, lattitude, longitude, state and postcode)
    # I also remove name column, as it contain more than 4500 different names,
    # and in the beggining I decide that different restaurant is restaurant with different coordinates, so we don't
    # need to use this information
    business = business.drop(columns=['categories','attributes','latitude','longitude',
                                      'state','postal_code','city','hours','is_open','review_count','name','address'])
    business = business.set_index(["business_id"])
    return  business

df_business_profiles = build_business_profiles(df_yelp_business)
df_business_profiles.sum()
df_business_profiles.dtypes

convenience stores             int64
tuscan                         int64
team building activities       int64
active life                    int64
cosmetics & beauty supply      int64
                              ...   
office cleaning                int64
pet stores                     int64
art galleries                  int64
soba                           int64
stars                        float64
Length: 412, dtype: object

In [13]:
def build_user_profiles(utility_matrix: pd.DataFrame, 
                                   business_profiles: pd.DataFrame) -> pd.DataFrame:
    # I will use something like TFIDF encoding, 
    # where I will add some value divided by total number of liked restaurant for specific user,
    # so it will be in range (-1,1) but then 
    # I will use normalization to get value from (0,1) as in business_profiles,
    # than I will also set stars to 5, as every user liked to visit restaurant with bigger review
    df = pd.DataFrame(0, index=utility_matrix.index, columns=business_profiles.columns,dtype=float)
    i = 0
    for user in utility_matrix.index:
        reviews = utility_matrix.loc[user]
        
        user_reviews = reviews[reviews != 0]
        visited_businesses = df_business_profiles[df_business_profiles.index.isin(list(user_reviews.index))]
        weighted_reviews = visited_businesses.mul(user_reviews, axis=0)
        weighted_reviews = weighted_reviews/len(user_reviews)
        weighted_reviews = weighted_reviews.sum(axis=0)
        df.loc[user] = weighted_reviews
        df.loc[user] = (df.loc[user] + 1)/2 
        df.at[user,'stars'] = 5
        i+=1
    return df
df_user_profiles = build_user_profiles(df_utility_matrix, df_business_profiles)


In [14]:
def predict_content_ratings(user_profiles: pd.DataFrame, business_profiles: pd.DataFrame) -> pd.DataFrame:
    # TODO: Distance based rating prediction (5 points)
    # TODO: Pointwise/Pairwase training based prediction (optional for 10 extra points)
    df = pd.DataFrame(0, index=user_profiles.index, columns=business_profiles.index,dtype=float)
    for user in user_profiles.index:
        user_df = user_profiles.loc[user]
        bussiness_df = business_profiles.sub(user_df,axis=1)
        business_df = bussiness_df.abs().sum(axis=1) #Manhetten distance
        # Normalize our values from 0 to 1
        mini = business_df.iloc[business_df.argmin()]
        maxi = business_df.iloc[business_df.argmax()] - mini
        business_df-=mini
        business_df/=maxi
        business_df = 1 - business_df

        df.loc[user] = business_df
    return df
df_content_predictions = predict_content_ratings(df_user_profiles, df_business_profiles)

In [15]:
import numpy as np
import time
import copy
def calculate_similarity(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    sim_matrix = pd.DataFrame(0,columns = utility_matrix.index,index=utility_matrix.index,dtype=int)
    x = 0
    t = time.time()
    sets = [set() for _ in range(utility_matrix.shape[0])]
    for i in utility_matrix.iterrows():
        ser = i[1]
        ar = np.array(ser)
        ar = ar.nonzero()
        sets[x] = set(copy.copy(list(ar[0])))
        x+=1
    n = utility_matrix.shape[0]
    t = time.time()
    for i in range(n):
        for j in range(i,n):
            k = list(sets[i] & sets[j])
            if k:
                k = (utility_matrix.iloc[i].iloc[k] == utility_matrix.iloc[j].iloc[k]).sum() 
                if k:
                    fst = utility_matrix.iloc[i].name
                    scd = utility_matrix.iloc[j].name
                    sim_matrix.at[fst,scd] = k
                    sim_matrix.at[scd,fst] = k
    return sim_matrix
sim_matrix = calculate_similarity(df_utility_matrix)

## Collaborative Filtering Recommender

In [16]:
def predict_collaborative_ratings(utility_matrix: pd.DataFrame) -> pd.DataFrame:
    # TODO: User-item collaborative filtering based rating prediction (15 points)
    # TODO: UV-decomposition based rating prediction (optional for 10 extra points)
    df = pd.DataFrame(0, index=utility_matrix.index, columns=utility_matrix.columns,dtype=float)
    for i in utility_matrix.iterrows():
        best = list(sim_matrix.loc[i[0]].nlargest(6).index)[1:]
        top_5 = utility_matrix.loc[best].sum(axis=0) / 5
        df.loc[i[0]] = top_5
    return df
df_collaborative_predictions = predict_collaborative_ratings(df_utility_matrix)

## Evaluation

In [17]:
import numpy as np
def score_model(utility_matrix: pd.DataFrame, predicted_utility_matrix: pd.DataFrame, model_name="model_0"):
    # TODO: Implement these by hand (each metric 1 point)
    rmse_score = np.sqrt(((predicted_utility_matrix - utility_matrix) *
                              (predicted_utility_matrix - utility_matrix)).sum().sum()
                                 /utility_matrix.shape[0] / utility_matrix.shape[1])
    map_score = np.mean(np.mean(np.abs((utility_matrix - predicted_utility_matrix))))
    coverage_score = 1 - np.mean(np.mean(utility_matrix == predicted_utility_matrix))
    personalization_score = 0
    intra_list_similarity_score = 0
    
    print("{} RMSE {}".format(model_name, rmse_score))
    print("{} MAP: {}".format(model_name, map_score))
    print("{} Coverage: {}".format(model_name, coverage_score))
    print("{} Personalization: {}".format(model_name, personalization_score))
    print("{} Intra-list similarity: {}".format(model_name, intra_list_similarity_score))    

score_model(df_content_predictions, df_utility_matrix, "content-based approach")
score_model(df_collaborative_predictions, df_utility_matrix, "collaborative-filtering approach")

content-based approach RMSE 0.5609759146392509
content-based approach MAP: 0.5307191173643095
content-based approach Coverage: 0.997968092472773
content-based approach Personalization: 0
content-based approach Intra-list similarity: 0
collaborative-filtering approach RMSE 0.019181355371861986
collaborative-filtering approach MAP: 0.0012088974553572172
collaborative-filtering approach Coverage: 0.009629975945889546
collaborative-filtering approach Personalization: 0
collaborative-filtering approach Intra-list similarity: 0


## Constraint Satisfaction Problem

We can work with the task of planing breakfast, lunch and dinner for particular user as Constraint Satisfaction Problem with

**Domain**: {all_businesses}

**Variables**: {breakfast, lunch, dinner}

**Constraints**: {constrainst regarding individual variable, or several variables at once}

We also have predicted ratings for every business and want to have personalized plan of restaurants. So we won't only satisfy our constraints, but also would like to get the maximum cumulative rating.

Take a look on prepared constraints and finish empty constraints in similar way (some of these constraints may require analytics on business data. e.g. to finish **has_coffee_constraint** you may need to determine all the categories which may include good coffee in their menu)

In [75]:
import ast
def is_vegetarian_constraint(business_id):
    try:
        return "vegetarian" in df_yelp_business[df_yelp_business["business_id"] == business_id].categories.values[0].lower()
    except:
        return False
def has_coffee_constraint(business_id):
    # TODO: implement this constraint (1 point)
    return "coffee" in df_yelp_business[df_yelp_business["business_id"] == business_id].categories.values[0].lower()


def has_alcohol_constraint(business_id):
    try: 
        return "alcohol" in df_yelp_business[df_yelp_business["business_id"] == business_id].attributes.values[0].lower()
    except:
        return False
def is_open_constraint(business_id):
    # TODO: implement this constraint (1 point)
    try:
        return df_yelp_business[df_yelp_business["business_id"] == business_id].is_open.values[0] == 1
    except:
        return False
    
def check_hour(time,time_str):
    time = int(time[:2]) * 60 + int(time[3:])
    time1 = int(time_str[:2]) * 60 + int(time_str[3:5])
    time2 = int(time_str[6:8]) * 60 + int(time_str[9:])
    return time1 < time < time2

def transform_weekday(weekday:str):
    return weekday[0].upper() + weekday[1:].lower()

def is_open_at_date_at_time_meta_constraint(weekday, time, business_id):
    # TODO: implement this constraint (1 point)
    weekday = transform_weekday(weekday)
    try:
        return is_open_constraint(business_id) and \
            check_hour(time,  ast.literal_eval(df_yelp_business[df_yelp_business["business_id"] == business_id].hours.values[0])[weekday])
    except:
        return False # Restaurant don't work at that time, or has Nan in hours

    
def is_open_at_monday_at_10am_constraint(business_id):
    return is_open_at_date_at_time_meta_constraint("monday", "10:00", business_id)

def all_are_different_constraint(state):
    for time in ["breakfast", "dinner", "lunch"]:
        for _t in ["breakfast", "dinner", "lunch"]:
            if time == _t: continue
            business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].categories.values[0].split(","))
            _business_categories = set(df_yelp_business[df_yelp_business["business_id"] == state[_t]["business_id"]].categories.values[0].split(","))
            if len(business_categories.intersection(_business_categories)) > \
                    len(business_categories.union(_business_categories)) // 2:
                return False
    return True
def all_are_in_the_same_city_constraint(state):
    cities = set()
    for time in ["breakfast", "dinner", "lunch"]:
        cities = cities | set(df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].city)
    return len(cities) < 2 # Zero or one
def all_are_in_the_same_region_meta_constraint(coordinates, threshold, state):
    # TODO: implement this constraint (1 point). Hint: use haversine distance https://pypi.org/project/haversine/
    now = True
    from haversine import haversine
    for time in ["breakfast", "dinner", "lunch"]:
            lat = df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].latitude.values[0]
            lon = df_yelp_business[df_yelp_business["business_id"] == state[time]["business_id"]].longitude.values[0]
            dst = haversine((float(lat),float(lon)),(coordinates['lat'],coordinates['lon']))
            now = now & (dst < threshold)
    return now
def all_are_in_test_region(state):
    return all_are_in_the_same_region_meta_constraint({"lat": 40.110446, "lon": -115.301568}, 600, state)

def at_least_one_visited_place_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one familiar place
    vis = 0
    now = 1/2
    for time in ["breakfast", "dinner", "lunch"]:
        if (df_utility_matrix.loc[state["user_id"]][state[time]["business_id"]] != 0):
            vis += now
            now/=2
    return vis

def at_least_one_has_coffee_constraint(state):
    # TODO: implement this constraint (2 points)
    # Make this constraint give more reward for more than one place with coffee
    coffee = 0
    now = 1/2
    for time in ["breakfast", "dinner", "lunch"]:
        if has_coffee_constraint(state[time]["business_id"]): # So now every time we find coffe our value will rise by one
            coffee += now
            now/=2

    return coffee


In [106]:
import random 

random.seed(42)
inspected_user = random.choice(df_yelp_reviews["user_id"].unique())

all_constraints = {
    "breakfast": [has_coffee_constraint, is_open_constraint],
    "lunch": [is_open_constraint],
    "dinner": [is_vegetarian_constraint, has_alcohol_constraint, is_open_constraint],
    "state": [at_least_one_has_coffee_constraint]
}

def goal_test(state: dict, constraints: dict):
    cumulative_rating = state["breakfast"]["predicted_rating"]*state["lunch"]["predicted_rating"]*\
                        state["dinner"]["predicted_rating"]
    for k in constraints.keys():
        if k == "state":
            for c in constraints[k]:
                cumulative_rating *= c(state)
        else:
            for c in constraints[k]:
                cumulative_rating *= c(state[k]["business_id"])
    return cumulative_rating

def init_filtering(domains,constraints,ratings):
    to_return = {
        'breakfast':[],
        'lunch':[],
        'dinner':[]
    }
    for time in to_return:
        for x in domains:
            correct = True
            for c in constraints[time]:
                if not c(x):
                    correct = False
                    break
            
            if correct and ratings[x]:                    # This value should have rating more than 0
                to_return[time].append(x)
                
    # Now ordering I will sort by rating, as it's impossible to predict future values
    
    for time in to_return:
        to_return[time].sort(key = lambda x: ratings[x])
    
    return to_return
best_score = 0
best_state = None
def backtracking(ubs,state,domain_filter,constraints,now):
    # We could use filtering only after we know lunch, as it possible to fit state constraint with only one last value
    # To be honest in this case there are no any advantages of using filtering(as we already use init_filtering)
    # but as it is need at task I will add it.

    for bs in domain_filter[now]:
        state[now]["business_id"] = bs
        state[now]['predicted_rating'] = ubs[bs]
        if now == 'breakfast':
            backtracking(ubs,state,domain_filter,constraints,'lunch')
        if now == 'lunch':
            copy_domain_filter = copy.deepcopy(domain_filter)
            copy_domain_filter['dinner'] = []
            for x in domain_filter['dinner']:
                correct = True
                state['dinner']["business_id"] = x
                state['dinner']['predicted_rating'] = ubs[x]
                for c in constraints['state']:
                    if not c(state):
                        correct = False
                if correct:
                    copy_domain_filter['dinner'].append(x)
                state['dinner']['business_id'] = None
                state['dinner']['predicted_rating'] = 0
                
            backtracking(ubs,state,copy_domain_filter,constraints,'dinner')
        if now == 'dinner':
            global best_score
            global best_state
            if goal_test(state,constraints) > best_score:
                best_score = goal_test(state,constraints)
                best_state = copy.deepcopy(state)
        state[now]["business_id"] = None
        state[now]['predicted_rating'] = 0

def prepare_restaurants_plan(user_id: str, user_business_ratings: pd.DataFrame, constraints: dict):
    # TODO: assign breakfast, lunch and dinner by solving Constraint Satisfaction Problem 
    # maximizing total score and satisfying all the constraints (it should work with any configuration of constraints)
    # You can implement Backtracking (10) + Filtering (10) + Ordering (5) using goal_test
    # OR
    # Local Search (10) + Min-Conflicts heuristic (10) + Ordering (5) with modification of goal test to work as Min-Conflicts heuristic
    state = {"user_id" : user_id,
        "breakfast": 
                {"business_id": None,
                 "predicted_rating": 0},
            "lunch": 
    {"business_id": None,
                 "predicted_rating": 0},
            "dinner": {"business_id": None,
                 "predicted_rating": 0}}
    domains = user_business_ratings.columns
    ratings = user_business_ratings.loc[user_id]
    domain_filter = init_filtering(domains,constraints,ratings)
    state = backtracking(ratings,state,domain_filter,constraints,'breakfast') # All other constraints already correct
    state = best_state
    state_v = goal_test(state, constraints)

    
    return state

# TODO: replace df_utility_matrix with your best predictions
state = prepare_restaurants_plan(inspected_user, df_collaborative_predictions, all_constraints)
print(state)
print(goal_test(state,all_constraints))

{'user_id': 'ZKrS8mZ23hBXTptp1P_qYA', 'breakfast': {'business_id': '1wAIVR71cLfupzNsk13Ryg', 'predicted_rating': 0.2}, 'lunch': {'business_id': 'eoJfl5vG7X87QhcKb0nt5Q', 'predicted_rating': 0.4}, 'dinner': {'business_id': 'zzSYBWuv_fXGtSgsO-6_1g', 'predicted_rating': 0.4}}
0.016000000000000004
